Készítsünk sorszámozott listát a szálláshelyek adatairól! 

a. A lista legyen a szálláshely típusa szerint, azon belül hely szerint, majd a neve szerint növekvően rendezve! 

b. A számozás típusonként kezdődjön újra!

In [ ]:
select *,ROW_NUMBER() over(PARTITION by tipus order by tipus,hely,szallas_nev)
 from  Szallashely

3

Készítsük listát, amely megjeleníti az ügyfelek azonosítóját, nevét és a foglalásainak kezdő- és befejező dátumát, valamint azt, hogy a foglalás összesen hány főre történt! 

a. Egy új oszlopban jelenítsük meg azt is, hogy az ügyfél előző három foglalása összesen hány főre történt!

In [ ]:
Select u.[LOGIN], u.NEV, f.METTOL, f.MEDDIG,f.FELNOTT_SZAM + f.GYERMEK_SZAM,
sum(f.FELNOTT_SZAM + f.GYERMEK_SZAM) OVER (PARTITION by u.LOGIN ORDER by f.mettol ROWS BETWEEN 3 Preceding and 1 Preceding)
from Foglalas f join  Ugyfel u on f.UGYFEL_FK=u.[LOGIN]

6

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">Alakítsa át a melléklet 1. pontjában lévő lekérdezést úgy, hogy JOIN helyett CROSS APPLY-t alkalmaz!</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Válaszként adja meg az átalakított lekérdezés kódját!</span>

In [ ]:
SELECT rt.TERMEKKOD, 
       AVG(rt.EGYSEGAR) AS 'Átlagos ár'
FROM Rendeles_tetel rt 
CROSS APPLY
(
    Select * from Rendeles r
     where rt.SORSZAM = r.SORSZAM
)r
GROUP BY rt.TERMEKKOD

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">7</span>

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">Alakítsa át a melléklet 2. pontjában lévő lekérdezést úgy, hogy mindkét (INNER) JOIN részbe sorra beilleszti a tanult JOIN HINT-eket (először mindkét helyre MERGE, majd HASH, utána LOOP, végül REMOTE).</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Melyik hint használata adja a legjobb megoldást?</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">b. Mi lesz ezt a hint-et tartalmazó lekérdezés kódja?</span>

In [ ]:
SELECT rt.*,
       u.nev,
       SUM(rt.mennyiseg*rt.egysegar) 
	   OVER(PARTITION BY r.LOGIN ORDER BY rt.SORSZAM, termekkod 
	     RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
	   AS 'Eddigi rendelési tételek összértéke'
FROM Rendeles_tetel rt Inner LOOP JOIN Rendeles r 
     ON rt.SORSZAM = r.SORSZAM
	 Inner LOOP JOIN Ugyfel u ON r.LOGIN = u.LOGIN

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">8</span>

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">Készítsen non-clustered indexet a Rendeles_tetel táblához!</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Az index neve NCI_sorszam_termekkod legyen</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">b. A sorszam legyen az index kulcs oszlop, a termékkód pedig az indexbe belefoglalt oszlop</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">c. Válaszként adja meg az indexet generáló script-et!</span>

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">9</span>

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">A mellékletben lévő 2. lekérdezéshez készítsen TABLE HINT-et a következőképpen:</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">a. Először állítsa be, hogy a lekérdezésnél a Rendeles_tetel tábla esetén a PK_Rendeles_tetel index legyen használva. Futtassa le így a lekérdezést, majd jegyezze fel a lekérdezés futási idejét!</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">b. Utána állítsa be, hogy a lekérdezésnél a Rendeles_tetel tábla esetén az NCI_sorszam_termekkod index legyen használva. Itt is futtassa a lekérdezéstm majd jegyezze meg a lekérdezés futási időt.</span>  
<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">c. Válaszként adja meg a gyorsabb változathoz tartozó lekérdezés kódját!</span>

In [ ]:
SELECT rt.*,
       u.nev,
       SUM(rt.mennyiseg*rt.egysegar) 
	   OVER(PARTITION BY r.LOGIN ORDER BY rt.SORSZAM, termekkod 
	     RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) 
	   AS 'Eddigi rendel�si t�telek �ssz�rt�ke'
FROM Rendeles_tetel rt JOIN Rendeles r 
     ON rt.SORSZAM = r.SORSZAM
	 JOIN Ugyfel u ON r.LOGIN = u.LOGIN
     Option (TABLE Hint(rt,INDEX(NCI_sorszam_termekkod)))

<span style="color: rgb(18, 38, 44); font-family: Arial; font-size: 15px; background-color: rgb(239, 249, 252);">Számítsa ki, hogy átlagosan hány napig tartottak a foglalások&nbsp;</span> <span style="box-sizing: border-box; color: rgb(18, 38, 44); font-family: Arial; font-size: 0.9375rem; background-color: rgb(239, 249, 252);">szállásonként, illetve ügyfelenként (külön-külön)!<br style="box-sizing: border-box;">A szállás,&nbsp;</span> <span style="box-sizing: border-box; color: rgb(18, 38, 44); font-family: Arial; font-size: 0.9375rem; background-color: rgb(239, 249, 252);">illetve az ügyfél esetén elég az azonosítójukat megadni.&nbsp;</span>

In [5]:
SELECT sz.szallas_fk, f.ugyfel_fk ,avg(DATEDIFF(day,f.METTOL,f.MEDDIG))  from Foglalas f join szoba sz on f.szoba_fk =sz.szoba_id
GROUP by GROUPING SETS(sz.SZALLAS_FK,f.UGYFEL_FK)

(207 rows affected)

Total execution time: 00:00:00.063

szallas_fk,ugyfel_fk,(No column name)
NULL,adam1,5
NULL,adam3,4
NULL,adam4,4
NULL,agnes,5
NULL,agnes3,4
NULL,AGNESH,4
NULL,AGNESK,2
NULL,akos,4
NULL,aladar,4
NULL,alexandra,4
